In [1]:
%cd ..

/tf/notebooks


In [2]:
import json
import re
import random

In [3]:
with open("data/fddb_data_v3_withimg.json", encoding='utf-8') as f:
    fddb = json.load(f)

In [4]:
with open("data/recipe-ingredient-to-fooddb.json") as f:
    all_ings_orig = json.load(f)

In [25]:
from extract_ingredients.util import normalize_out_ingredient


In [31]:
custom_amounts = {("Öl", "Olivenöl"): ["UNK (20 g)", "etwas (20 g)"], # 2EL = 30g
                  ("Salz",): ["UNK (2 g)"],
                 }

custom_amounts = {ks: v for k, v in custom_amounts.items() for ks in k}
custom_amounts

{'Olivenöl': ['UNK (20 g)', 'etwas (20 g)'],
 'Salz': ['UNK (2 g)'],
 'Öl': ['UNK (20 g)', 'etwas (20 g)']}

In [6]:
from collections import defaultdict
fddb_entries = defaultdict(list)
rem = re.compile(r"^(\d+ )?(.*) \((\d+(?:,\d+)?) (g|ml)\)$")
def handle_fddb():
    for e in fddb:
        for ning in normalize_out_ingredient(e["name"]):
            fddb_entries[ning].append(e)
        sanity = list(e["Standard Nährwerte"].keys())
        snmatch = re.match("^Nährwerte für 100 (\w+)$", sanity[0])
        if snmatch is not None:
            unn = snmatch.groups()[0]
            e["parsed"] = {"unit": unn, "for_100": e["Standard Nährwerte"]["Nährwerte für 100 {}".format(unn)]}

        else:
            print("SANITY ERROR:", e["Id"], sanity, unit)
            continue
        amts = e["parsed"]["amounts"] = []
        for k in e["Spezifische Nährwerte"].keys():
            if k == "100 g (100 g)" or k == "100 g (100 ml)" or k == "100 ml (100 ml)":
                continue
            txtamount, txt, amount, unit = rem.match(k).groups()
            amount = float(amount.replace(",", "."))
            txtamount = float(txtamount.replace(",", ".")) if txtamount else 1.0
            amts.append((txt, k, amount/txtamount))

            if unn != unit:
                raise Exception("noooo")
            #print(txt, amount/txtamount, unit)
handle_fddb()

SANITY ERROR: https://fddb.info/db/de/lebensmittel/selbst_gemacht_100_kalorien_dummy/index.html ['Nährwerte deines Gerichts kennst und in dein Ernährungstagebuch eintragen möchtest.\n31 Bewertungen\nSchreibe eine Bewertung\nHersteller: Selbst gemacht\nProduktgruppe: Sonstiges\nDatenquelle: Schätzung. Die Produktdaten wurden am 18.05.2006 von einem Fddb Nutzer erhoben. Hinweise zu den Produktdaten. Aktualisiert: 08.11.2018. '] g
SANITY ERROR: https://fddb.info/db/de/lebensmittel/weider_weider_yippie_riegel_cookies_netto_naehrwerte/index.html ['Nährwerte) von MarieCarlotta | Hochgeladen von: MarieCarlotta (Problem melden) Details\n\nFoto hochladen\nNoch keine Beschreibung für dieses Produkt.\nAngaben noch nicht bestätigt. Melde einen Fehler oder korrigiere die Angaben.\nSchreibe eine Bewertung\nHersteller: Weider\nProduktgruppe: Nahrungsergänzung\nDatenquelle: Extern. Die Produktdaten wurden am 09.11.2017 von einem Fddb Nutzer erhoben. Hinweise zu den Produktdaten. Aktualisiert: 04.07.20

In [7]:
# get recipe meta
with open("data/recipes/processed_data.json", encoding="utf-8") as f:
    recipes = json.load(f)

In [8]:
# get amount map from parse-amounts
with open("data/recipes/parsed_amounts.json") as f:
    parsed_amounts = json.load(f)

In [9]:
# only recipes with pictures
recipes = [recipe for recipe in recipes if len(recipe['picture_files']) > 0]

In [24]:
from operator import itemgetter
import pandas as pd
pd.set_option('display.max_colwidth', -1)
from IPython.display import HTML
from html import escape
# ugly af code
debug=False

def match_amts(*, ing_matches, have_count, have_unit):
    possible_entries = {entry["Id"]: (ing_match[1], entry) for ing_match in ing_matches for entry in fddb_entries[ing_match[0]]}
    possible_entries = sorted(list(possible_entries.values()), key=lambda e: -e[0])
    for accuracy, e in possible_entries:
        if "parsed" not in e:
            continue
        debug and print("considering", e["name"], e["Id"])
        want_unit = e["parsed"]["unit"]

        if want_unit in have_unit:
            debug and print("got direct match!", have_count, have_unit, have_count, want_unit)
            return (e, accuracy, have_count, want_unit, have_count, want_unit)
        for amount_name, amount_source, to_norm in e["parsed"]["amounts"]:
            debug and print(e["name"], "amount", amount_source)
            if amount_name in have_unit:
                debug and print("got match unit!", amount_source)
                return (e, accuracy, have_count, have_unit[0], have_count*to_norm, want_unit)
            
            
def match_ingredient(ingredient):
    cols = []
    if "amount" in ingredient:
        cols += [ingredient["amount"], ingredient["ingredient"], "="]
        amount_count, amount_type = parsed_amounts[ingredient["amount"]]

        if amount_count is None:
            return cols + ["[ignored]"]
        if amount_type is None:
            amount_type = ["Stück", "mittelgroß", "mittelgroße", "mittelgroßes", "mittelgroßer"]
        elif amount_type in ["dicke", "dicker"]:
            amount_type = [amount_type, "große", "großer"]
        else:
            amount_type = [amount_type]
        #print("[" + str(amount_count) + "*" + (amount_type)+ "] of ", end="")
        _ing_matches = all_ings_orig[ingredient["ingredient"]]
        ing_matches = [e for e in _ing_matches if e[1] > 0.85]
        if len(ing_matches) == 0:
            # not so great match but eh
            ing_matches = [_ing_matches[0]]


        debug and print("matches", ing_matches)
        debug and print("finding unit ", amount_type)
        match = match_amts(ing_matches=ing_matches, have_count=amount_count, have_unit=amount_type)

        if not match:
            cols.append("[no amount for match for [{}] on [{}]]".format("|".join(amount_type), "|".join(t for t, _ in ing_matches)))
        else:
            fddb_entry, accuracy, count_weird, unit_weird, count_norm, unit_norm = match
            if unit_weird == unit_norm:
                unittxt = "{}{}".format(count_norm, unit_norm)
            else:
                unittxt = "{} {} = {}{}".format(count_weird, unit_weird, count_norm, unit_norm)
            values = fddb_entry["parsed"]["for_100"]
            value_kcal_per_100 = float(values["Kalorien"]["Menge"].replace(",", "."))
            value_kcal = value_kcal_per_100 / 100 * count_norm
            cols.append("({:.0f}%) [{}] of {} = {:.0f} kcal".format(accuracy * 100, unittxt, fddb_entry["name"], value_kcal))
        return cols
    else:
        return ["->" + ingredient["subtitle"], "", "", ""]
def match_recipe(recipe):
    display(HTML("<h2>{}</h2>".format(escape(recipe["title"]))))
    ings = []
    for ingredient in recipe["ingredients"]:
        ings.append(match_ingredient(ingredient))
    ings_df = pd.DataFrame(ings, columns=["inp amount", "inp ing", "", "paredd"])
    display(HTML(ings_df.to_html()))
for recipe in random.sample(recipes, 20):
    match_recipe(recipe)

,inp amount,inp ing,,paredd
0,300 g,"Hackfleisch, gemischtes",=,(100%) [300g] of Hackfleisch = 360 kcal
1,300 g,"Gulasch, gemischt",=,(100%) [300g] of Gulasch = 408 kcal
2,600 g,Kartoffel(n),=,(100%) [600g] of Kartoffeln = 462 kcal
3,300 g,Möhre(n),=,(97%) [300g] of Möhrchen = 69 kcal
4,2,Zwiebel(n),=,"(96%) [2 Stück = 170.0g] of Zwiebel, frisch = 48 kcal"
5,500 g,"Tomate(n), passierte",=,"(100%) [500g] of Tomaten, frisch = 90 kcal"
6,2 EL,Tomatenmark,=,"[no amount for match for [Esslöffel] on [Tomatenmark|Tomaten-Mark|Tomatenpüree|Tomatensosse|Tomaten|Tomatenmark, scharf|Tomatenreis|Tomatenstücke|Tomaten Stückig|Stückige Tomaten]]"
7,1 Handvoll,"Oregano, frischer",=,"[no amount for match for [Handvoll] on [Oregano|Oliven Oregano|Oregano, frisch|Oregano, getrocknet]]"
8,3 EL,Worcestersauce,=,"[no amount for match for [Esslöffel] on [Worcester Sauce|Worcestershire Sauce|Worcestershire Sauce, Würzsauce]]"
9,,Salz,=,(91%) [1 Stück = 35.0g] of Salzgurken = 5 kcal


,inp amount,inp ing,,paredd
0,500 ml,Milch,=,(100%) [500ml] of Milch = 275 kcal
1,40 g,Speisestärke,=,(100%) [40g] of Speisestärke = 140 kcal
2,2 Stück(e),"Banane(n), (ca. 200 g geschält)",=,(96%) [2 Stück = 10.0g] of Bananas = 38 kcal
3,120 g,Nougat,=,(100%) [120g] of Nougat = 686 kcal
4,n. B.,"Zucker, 1-2 EL",=,[ignored]


,inp amount,inp ing,,paredd
0,8,Hähnchenunterkeule(n),=,[no amount for match for [Stück|mittelgroß|mittelgroße|mittelgroßes|mittelgroßer] on [Hähnchenunterkeulen|Hähnchenkeulen|Hähnchen Unterkeulen|Hähnchen Oberkeulen|Hähnchen-Unter-Keulen|Hähnchengeschnezeltes|Hähnchenmagen|frische Hähnchenunterkeulen|Hähnchenoberkeulenspieß|Hähnchen-Sülze]]
1,200 g,Lauch,=,(89%) [200g] of Lauchschinken = 460 kcal
2,200 g,Zwiebel(n),=,(100%) [200g] of Zwiebeln = 68 kcal
3,200 g,Frühlingszwiebel(n),=,(94%) [200g] of Silberzwiebeln = 108 kcal
4,n. B.,"Gewürzmischung, arabische (Baharat)",=,[ignored]
5,n. B.,Öl,=,[ignored]


,inp amount,inp ing,,paredd
0,400 g,"Rindfleisch, mager (z. B. Rinderbrust)",=,(100%) [400g] of Rindfleisch = 712 kcal
1,2 m.-große,"Zwiebel(n), gehackt",=,"(96%) [2 mittelgroße = 170.0g] of Zwiebel, frisch = 48 kcal"
2,1,"Knoblauchzehe(n), gehackt",=,(93%) [1 Stück = 4.0g] of Knoblauchpüree = 6 kcal
3,3 EL,Tomatenmark,=,"[no amount for match for [Esslöffel] on [Tomatenmark|Tomaten-Mark|Tomatenpüree|Tomatensosse|Tomaten|Tomatenmark, scharf|Tomatenreis|Tomatenstücke|Tomaten Stückig|Stückige Tomaten]]"
4,1 TL,Chilisalz,=,[no amount for match for [Teelöffel] on [Knoblauchsalz|Grillgewürzsalz|Chilischote|Chilisoße]]
5,1 TL,"Paprikapulver, scharf",=,[no amount for match for [Teelöffel] on [Paprikapulver|Paprikagulasch|Paprikacreme|Paprika-Creme|Paprika Quark|Paprikasoße|Paprikasülze|Paprika-Mandel-Dip|Paprika-Lyoner|Paprikasuppe]]
6,1 TL,"Paprikapulver, süß",=,[no amount for match for [Teelöffel] on [Paprikapulver|Paprikagulasch|Paprikacreme|Paprika-Creme|Paprika Quark|Paprikasoße|Paprikasülze|Paprika-Mandel-Dip|Paprika-Lyoner|Paprikasuppe]]
7,,Pfeffer,=,(88%) [1 Stück = 87.0g] of Pfefferbreze = 276 kcal
8,1 Glas,Rotwein,=,(94%) [1 Glas = 200.0ml] of Rotwein trocken = 170 kcal
9,300 ml,Fleischbrühe,=,(96%) [300ml] of Rinderbrühe = 81 kcal


,inp amount,inp ing,,paredd
0,500 m.-große,"Champignons, frische weiße",=,[no amount for match for [mittelgroße] on [Champignons|champignons|Pfannen-Champignons|Champignon|Rahmchampignons|Rahm-Champignons|Champignons gebraten|Chapignon|Braune Champignons|Gefüllte Champignons]]
1,1,"Zitrone(n), Saft davon",=,(93%) [1 Stück = 70.0g] of Zitronenrolle = 193 kcal
2,1 Bund,"Petersilie, glatte, sehr fein gehackt",=,"(88%) [1 Bund = 35.0g] of Petersilie, frisch = 18 kcal"
3,2,"Knoblauchzehe(n), klein gewürfelt oder mit Salz zerrieben",=,(93%) [2 Stück = 8.0g] of Knoblauchpüree = 11 kcal
4,125 ml,"Olivenöl, bestes",=,(100%) [125ml] of Olivenöl = 1035 kcal
5,2,"Schalotte(n), klein gewürfelt oder mit Salz zerrieben",=,(86%) [2 Stück = 240.0g] of Maisecke = 809 kcal


...

,inp amount,inp ing,,paredd
0,700 g,Möhre(n),=,(97%) [700g] of Möhrchen = 161 kcal
1,1 große,Süßkartoffel(n),=,[no amount for match for [große] on [Süßkartoffeln|Bratkartoffeln|Ofenkartoffeln|Butterkartoffeln|Süßkartoffel|Speisekartoffeln|Süßkartoffelpürree|bratkartoffeln|Schmorkartoffeln|Stampfkartoffeln]]
2,1,Zwiebel(n),=,"(96%) [1 Stück = 85.0g] of Zwiebel, frisch = 24 kcal"
3,1,Knoblauchzehe(n),=,(93%) [1 Stück = 4.0g] of Knoblauchpüree = 6 kcal
4,2 cm,Ingwer,=,"[no amount for match for [cm] on [Ingwer|Ingwer pur|Ingwer, kandiert|Ingwersaft|Ingwer-Zitrone|Ingwer Saft|Ingwer Tee]]"
5,2 TL,Garam Masala oder Curry,=,[no amount for match for [Teelöffel] on [Garam Masala|Kolhapuri Masala|Tikka Masala|Masala Chai|Basmati Reis|Garam Masala Brotaufstrich|Tikka Masala Paste|Chicken Tikka Masala|Curry Masala]]
6,1200 ml,Gemüsebrühe,=,(100%) [1200ml] of Gemüsebrühe = 36 kcal
7,200 ml,Kokosmilch,=,(100%) [200ml] of Kokosmilch = 358 kcal
8,,Salz und Pfeffer,=,"[no amount for match for [Stück|mittelgroß|mittelgroße|mittelgroßes|mittelgroßer] on [Pfeffer & Salz Chips, Pfeffer und Salz]]"
9,2 EL,Öl,=,[no amount for match for [Esslöffel] on [Öl]]


,inp amount,inp ing,,paredd
0,700 g,Apfelmus,=,(100%) [700g] of Apfelmus = 518 kcal
1,4,Ei(er),=,"(100%) [4 Stück = 220.0g] of Ei, vom Huhn = 301 kcal"
2,200 g,Butter,=,(100%) [200g] of Butter = 1486 kcal
3,500 g,Mehl,=,(100%) [500g] of Mehl = 1740 kcal
4,2 TL,Backpulver,=,(100%) [2 Teelöffel = 6.0g] of Backpulver = 6 kcal
5,160 g,Zucker,=,(100%) [160g] of Zucker = 640 kcal
6,100 g,Speisestärke,=,(100%) [100g] of Speisestärke = 351 kcal
7,,Zimt und Zucker zum Bestreuen,=,"(91%) [1 Stück = 6.3g] of Zimtsterne, Zimt = 31 kcal"


,inp amount,inp ing,,paredd
0,200 g,Milchreis,=,(100%) [200g] of Milchreis = 702 kcal
1,1 Liter,Milch,=,(100%) [1000ml] of Milch = 550 kcal
2,1 Glas,Sauerkirschen,=,(96%) [1 Glas = 185.0g] of Süßkirschen = 74 kcal
3,200 g,"Mandel(n), gehackt",=,(100%) [200g] of Mandeln = 1260 kcal
4,2 Becher,Sahne,=,(100%) [2 Becher = 400.0ml] of Sahne = 1172 kcal
5,50 g,Zucker,=,(100%) [50g] of Zucker = 200 kcal
6,1 Prise(n),Salz,=,(100%) [1 Prise = 1.0g] of Salz = 0 kcal
7,25 g,Speisestärke,=,(100%) [25g] of Speisestärke = 88 kcal
8,etwas,Vanille,=,[ignored]
9,n. B.,Zimt - Zucker,=,[ignored]


,inp amount,inp ing,,paredd
0,2,Lachs,=,(89%) [2 Stück = 200.0g] of Lachsforelle = 312 kcal
1,2,Schalotte(n),=,(86%) [2 Stück = 240.0g] of Maisecke = 809 kcal
2,400 g,"Nudeln, nach Wahl",=,(100%) [400g] of Nudeln = 1052 kcal
3,50 g,Parmesan,=,(89%) [50g] of parmesan = 201 kcal
4,1 EL,Butter,=,(85%) [1 Esslöffel = 12.0g] of Butterschmalz = 106 kcal
5,1 Becher,saure Sahne,=,(92%) [1 Becher = 200.0ml] of Sahne = 586 kcal
6,1 Prise(n),Muskatnuss,=,"(90%) [1 Prise = 0.1g] of Muskat, gemahlen = 1 kcal"
7,1 Prise(n),Pfeffer,=,[no amount for match for [Prise] on [Pfefferonen|Pfefferbreze|Pfefferminze|Pfeffer-Braten|Pfefferbeiser|Pfeffer-Beißer|Pfefferminztee|Pfefferbeisser|Pfeffersäckchen|Bunter Pfeffer]]
8,1 Prise(n),Salz,=,(100%) [1 Prise = 1.0g] of Salz = 0 kcal


,inp amount,inp ing,,paredd
0,3,Salatgurke(n),=,[no amount for match for [Stück|mittelgroß|mittelgroße|mittelgroßes|mittelgroßer] on [Salatgurke|gurkensalat|Jungsalat|Königssalat|Randensalat|Hirtensalat|Blattsalat|Kopfsalat|Feldsalat|Heringsalat]]
1,600 g,Schweinefilet(s),=,(96%) [600g] of Schweinefilet = 690 kcal
2,1 große,Zwiebel(n),=,"(96%) [1 große = 250.0g] of Zwiebel, frisch = 70 kcal"
3,200 g,Sahne,=,(93%) [200g] of Saure Sahne = 238 kcal
4,400 ml,Wasser,=,(100%) [400ml] of Wasser = 4 kcal
5,,"Öl, zum Braten",=,[no amount for match for [Stück|mittelgroß|mittelgroße|mittelgroßes|mittelgroßer] on [Öl]]
6,1 Bund,Dill,=,[no amount for match for [Bund] on [Dill]]
7,,Gemüsebrühwürfel,=,"(93%) [1 Stück = 10.0g] of Gemüsebrühe, Würfel = 1 kcal"
8,,"Salz und Pfeffer, zum Würzen",=,"[no amount for match for [Stück|mittelgroß|mittelgroße|mittelgroßes|mittelgroßer] on [Pfeffer & Salz Chips, Pfeffer und Salz]]"
9,1 Prise(n),Zucker,=,(100%) [1 Prise = 1.0g] of Zucker = 4 kcal


In [ ]:
fddb_entries["Ei, vom Huhn"]

In [ ]:
normalize_out_ingredient("Ei, vom Huhn")

In [14]:
match_ingredient({"amount": "1 Pck.", "ingredient": "Backpulver"})

matches [['Backpulver', 1.0000001192092896], ['Volleipulver', 0.9295499324798584], ['Basenpulver', 0.8921348452568054], ['Vanillepulver', 0.8914501667022705], ['Getränkepulver', 0.8904077410697937], ['Macapulver', 0.8899227380752563], ['Kakaopulver', 0.8799252510070801], ['Magermilchpulver', 0.873859167098999], ['Reinweinstein-Backpulver', 0.8718467950820923], ['Back Kakaopulver', 0.8572609424591064]]
finding unit  ['Packung']
considering Backpulver https://fddb.info/db/de/lebensmittel/ruf_backpulver/index.html
Backpulver amount 1 Packung (15 g)
got match unit! 1 Packung (15 g)
kcalvals 87.0 15.0


['1 Pck.',
 'Backpulver',
 '=',
 '(100%) [1 Packung = 15.0g] of Backpulver = 13 kcal']

In [ ]:
z = fddb_entries["Backpulver"][2]["parsed"]["for_100"]["Kalorien"]["Menge"]
float(z.replace(",", "."))

In [18]:
match_ingredient({"amount": "1", "ingredient": "Schalotte(n)"})

matches [['Schalotten', 1.0000001192092896], ['Sülze', 0.8927401900291443], ['Ringlotten', 0.8788649439811707], ['Schaschlik', 0.8767236471176147], ['Leberkse', 0.8710306882858276], ['Schwartenmagen', 0.8679845333099365], ['Maiskölbchen', 0.8627955913543701], ['Grützwust', 0.8598487377166748], ['Maisecke', 0.8587009310722351], ['Käsknacker', 0.8583810329437256]]
finding unit  ['Stück', 'mittelgroß', 'mittelgroße', 'mittelgroßes', 'mittelgroßer']
considering Schalotten https://fddb.info/db/de/lebensmittel/aldi_schalotten/index.html
Schalotten amount Netz (500 g)
considering Sülze https://fddb.info/db/de/lebensmittel/metzger_suelze/index.html
Sülze amount 1 Portion (50 g)
considering Ringlotten https://fddb.info/db/de/lebensmittel/naturprodukt_ringlotten/index.html
Ringlotten amount 1 Portion (100 g)
considering Schaschlik https://fddb.info/db/de/lebensmittel/selbst_gemacht_schaschlik/index.html
Schaschlik amount 1 Portion (150 g)
considering Leberkse https://fddb.info/db/de/lebensmittel

['1', 'Schalotte(n)', '=', '(86%) [1 Stück = 120.0g] of Maisecke = 404 kcal']

In [21]:
match_ingredient({"amount": "1 Glas", "ingredient": "Schattenmorellen"})

matches [['Schattenmorellen', 0.9999998807907104], ['Schattenmorellen entsteint', 0.9361054301261902], ['Schattenmorellen, entsteint', 0.9014579653739929], ['Natreen Schattenmorellen', 0.8644647002220154]]
finding unit  ['Glas']
considering Schattenmorellen https://fddb.info/db/de/lebensmittel/edeka_schattenmorellen/index.html
considering Schattenmorellen https://fddb.info/db/de/lebensmittel/schneekoppe_schattenmorellen/index.html
Schattenmorellen amount 1 Glas (340 g)
got match unit! 1 Glas (340 g)
kcalvals 42.0 340.0


['1 Glas',
 'Schattenmorellen',
 '=',
 '(100%) [1 Glas = 340.0g] of Schattenmorellen = 143 kcal']